In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./examples/oj.csv")

In [3]:
df.head()

,sales,price,brand,feat
0,8256.0,3.87,tropicana,0
1,6144.0,3.87,tropicana,0
2,3840.0,3.87,tropicana,0
3,8000.0,3.87,tropicana,0
4,8896.0,3.87,tropicana,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28947 entries, 0 to 28946
Data columns (total 4 columns):
sales    28947 non-null float64
price    28947 non-null float64
brand    28947 non-null object
feat     28947 non-null int64
dtypes: float64(2), int64(1), object(1)
memory usage: 904.7+ KB
